<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas_postgres/pandas_postgres_hard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

185. Department Top Three Salaries

https://leetcode.com/problems/department-top-three-salaries/description/

```sql

SELECT d.name as Department, sub.name as Employee, sub.salary as Salary
FROM (
SELECT
    e.departmentId , e.name, e.salary,
    dense_rank() OVER (PARTITION BY departmentId ORDER BY salary DESC) as rnk
FROM Employee e) sub
LEFT JOIN Department d ON d.id = sub.departmentId
WHERE sub.rnk < 4
```

In [ ]:
data = [[1, 'Joe', 85000, 1], [2, 'Henry', 80000, 2], [3, 'Sam', 60000, 2], [4, 'Max', 90000, 1], [5, 'Janet', 69000, 1], [6, 'Randy', 85000, 1], [7, 'Will', 70000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    return employee.assign(rnk = employee.groupby('departmentId').salary.rank(method='dense', ascending=False))\
                    .query('rnk <= 3', engine='python')\
                    .merge(department, how='left', left_on='departmentId', right_on='id', suffixes=['_emp', '_dpt'])\
                    [['name_dpt', 'name_emp', 'salary']].rename({'name_dpt': 'Department', 'name_emp': 'Employee', 'salary': 'Salary'}, axis=1)

262. Trips and Users

https://leetcode.com/problems/trips-and-users/description/

```sql
WITH banned AS (
    SELECT users_id FROM Users WHERE banned = 'Yes'
)
SELECT
request_at as Day,
ROUND(1 - count(id) FILTER (WHERE t.status NOT IN ('cancelled_by_driver', 'cancelled_by_client'))*1.0 / count(id), 2) as "Cancellation Rate"

FROM Trips t
WHERE t.request_at  BETWEEN '2013-10-01' and '2013-10-03' AND t.client_id NOT IN (SELECT users_id FROM banned) AND t.driver_id NOT IN (SELECT users_id FROM banned)
GROUP BY request_at
HAVING count(t.id) > 0
```

The cancellation rate is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Write a solution to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03" with at least one trip. Round Cancellation Rate to two decimal points.

Return the result table in any order.



In [ ]:
data = [['1', '1', '10', '1', 'completed', '2013-10-01'], ['2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01'], ['3', '3', '12', '6', 'completed', '2013-10-01'], ['4', '4', '13', '6', 'cancelled_by_client', '2013-10-01'], ['5', '1', '10', '1', 'completed', '2013-10-02'], ['6', '2', '11', '6', 'completed', '2013-10-02'], ['7', '3', '12', '6', 'completed', '2013-10-02'], ['8', '2', '12', '12', 'completed', '2013-10-03'], ['9', '3', '10', '12', 'completed', '2013-10-03'], ['10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03']]
trips = pd.DataFrame(data, columns=['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at']).astype({'id':'Int64', 'client_id':'Int64', 'driver_id':'Int64', 'city_id':'Int64', 'status':'object', 'request_at':'object'})

data = [['1', 'No', 'client'], ['2', 'Yes', 'client'], ['3', 'No', 'client'], ['4', 'No', 'client'], ['10', 'No', 'driver'], ['11', 'No', 'driver'], ['12', 'No', 'driver'], ['13', 'No', 'driver']]
users = pd.DataFrame(data, columns=['users_id', 'banned', 'role']).astype({'users_id':'Int64', 'banned':'object', 'role':'object'})

In [ ]:
def trips_and_users(trips: pd.DataFrame, users: pd.DataFrame) -> pd.DataFrame:
    banned_ids = users[users.banned == 'Yes'].users_id.values
    df = trips[(trips.request_at.between('2013-10-01', '2013-10-03')) &
        (~trips.client_id.isin(banned_ids)) &
        (~trips.driver_id.isin(banned_ids))]
    df = df.assign(cancelled = np.where(df.status.str.match('cancelled'), 1, 0))\
    .groupby('request_at')['cancelled'].agg(['sum', 'size']).reset_index().rename({"request_at": "Day"}, axis=1)


    df['Cancellation Rate'] = (df['sum']/df['size']).round(2)
    return df[['Day','Cancellation Rate']]

601. Human Traffic of Stadium

https://leetcode.com/problems/human-traffic-of-stadium/description/

```sql
-- Write your PostgreSQL query statement below
SELECT
id, visit_date, people
FROM (
SELECT
*,
LEAD(id) OVER (ORDER BY id) as ld,
LAG(id) OVER (ORDER BY id) as lg,
LEAD(id, 2) OVER (ORDER BY id) as ld2,
LAG(id, 2) OVER (ORDER BY id) as lg2
FROM Stadium
WHERE people   >= 100) sub
WHERE (id - lg = 1 AND ld - id = 1) OR
      (id - lg = 1 AND id - lg2 = 2) OR
      (ld - id = 1 AND ld2 - id = 2)
ORDER BY visit_date
```

Write a solution to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

The result format is in the following example.

In [ ]:
data = [[1, '2017-01-01', 10], [2, '2017-01-02', 109], [3, '2017-01-03', 150], [4, '2017-01-04', 99], [5, '2017-01-05', 145], [6, '2017-01-06', 1455], [7, '2017-01-07', 199], [8, '2017-01-09', 188]]
stadium = pd.DataFrame(data, columns=['id', 'visit_date', 'people']).astype({'id':'Int64', 'visit_date':'datetime64[ns]', 'people':'Int64'})

In [ ]:
def human_traffic(stadium: pd.DataFrame) -> pd.DataFrame:
    filtered = stadium[stadium.people.ge(100)]
    return filtered.assign(lg1_diff = filtered.id.diff(1), ld1_diff = - filtered.id.diff(-1), lg2_diff = filtered.id.diff(2), ld2_diff = - filtered.id.diff(-2))\
                  .query("(lg1_diff == 1 & ld1_diff == 1) | \
                          (ld1_diff == 1 & ld2_diff == 2) | \
                          (lg1_diff == 1 & lg2_diff == 2)", engine='python')\
                          .iloc[:,0:3].sort_values(by='visit_date')

3374. First Letter Capitalization II

https://leetcode.com/problems/first-letter-capitalization-ii/description/

```sql
SELECT
	content_id,
	content_text AS original_text,
  initcap(content_text)   AS converted_text
FROM user_content
```

In [ ]:
data = [[1, 'hello world of SQL'], [2, 'the QUICK-brown fox'], [3, 'modern-day DATA science'], [4, 'web-based FRONT-end development']]
user_content = pd.DataFrame(data, columns=['content_id', 'content_text'])

In [ ]:
def capitalize_content(user_content: pd.DataFrame) -> pd.DataFrame:
    user_content['converted_text'] = user_content.content_text.apply(lambda x: x.title())
    return user_content.rename({'content_text': 'original_text'}, axis=1)

569. Median Employee Salary

https://leetcode.com/problems/median-employee-salary/description/

```sql
SELECT id, company, salary FROM (
SELECT
company,
id, salary,
row_number() OVER (PARTITION BY company ORDER BY salary DESC, id DESC) -
row_number() OVER (PARTITION BY company ORDER BY salary ASC, id ASC) as rn_delta,
count(*) OVER (PARTITION BY company) AS  cnt
FROM Employee
 ) sub
--WHERE rn_delta <= 1 - (cnt * 1.0) % 2 AND rn_delta >= -1 + (cnt * 1.0) % 2
WHERE rn_delta <= 1  AND rn_delta >= -1
```

571. Find Median Given Frequency of Numbers

https://leetcode.com/problems/find-median-given-frequency-of-numbers/description/

```sql
SELECT
percentile_cont(0.5) WITHIN GROUP (ORDER BY num) as median FROM (
SELECT num, generate_series(1, frequency)  FROM Numbers) sub
```

579. Find Cumulative Salary of an Employee

https://leetcode.com/problems/find-median-given-frequency-of-numbers/

```sql
WITH t as (
SELECT
id, generate_series(1, mm) as month
FROM (
SELECT
id, max(month) - 1 as mm
FROM Employee
GROUP BY id) sub
ORDER BY 1, 2),

analytical_t as (
SELECT
id, month, COALESCE(salary, 0) as salary
FROM t
LEFT JOIN Employee USING (id, month)),

full_table as (
SELECT
id, month,
sum(salary) OVER (PARTITION BY id ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as Salary
FROM analytical_t)

SELECT f.id, f.month, f.Salary FROM full_table f
INNER JOIN Employee e USING(id, month)
ORDER BY 1, 2 DESC
```

```sql
SELECT e.id, e.month,
    SUM(e2.Salary) as Salary
FROM Employee e
JOIN Employee e2 ON (e.Id = e2.Id AND e.Month >= e2.Month AND (e.Month - e2.Month <= 2))
WHERE e.Month < (SELECT MAX(Month) from Employee where Id = e.Id)
GROUP BY e.Id, e.Month
ORDER BY e.Id, e.Month DESC
```

1336. Number of Transactions per Visit

https://leetcode.com/problems/number-of-transactions-per-visit/description/

```sql
WITH t as (
SELECT
v.user_id,
v.visit_date,
count(v.visit_date) as visit_count,
count(t.amount) as trancations_count
FROM Visits v
LEFT JOIN Transactions t
ON v.user_id = t.user_id AND  v.visit_date = t.transaction_date
GROUP BY v.user_id, v.visit_date),

t_cnts AS (
SELECT
generate_series(0, max(trancations_count)) as trancations_count_full
FROM t
)

SELECT
trancations_count_full as transactions_count,
count(t.user_id) as visits_count
FROM t_cnts
LEFT JOIN t ON t.trancations_count = t_cnts.trancations_count_full
GROUP BY trancations_count_full
ORDER BY 1
```

In [ ]:
data = [[1, '2020-01-01'], [2, '2020-01-02'], [12, '2020-01-01'], [19, '2020-01-03'], [1, '2020-01-02'], [2, '2020-01-03'], [1, '2020-01-04'], [7, '2020-01-11'], [9, '2020-01-25'], [8, '2020-01-28']]
visits = pd.DataFrame(data, columns=['user_id', 'visit_date']).astype({'user_id':'Int64', 'visit_date':'datetime64[ns]'})
data = [[1, '2020-01-02', 120], [2, '2020-01-03', 22], [7, '2020-01-11', 232], [1, '2020-01-04', 7], [9, '2020-01-25', 33], [9, '2020-01-25', 66], [8, '2020-01-28', 1], [9, '2020-01-25', 99]]
transactions = pd.DataFrame(data, columns=['user_id', 'transaction_date', 'amount']).astype({'user_id':'Int64', 'transaction_date':'datetime64[ns]', 'amount':'Int64'})

In [ ]:
transactions_in_visits = visits.merge(transactions, how='left', left_on=['user_id', 'visit_date'], right_on=['user_id', 'transaction_date']).groupby(['user_id', 'visit_date'], as_index=False).amount.count()
all_t_cnt = pd.Series(list(range(max(transactions_in_visits.amount) + 1)), name='tcnt').to_frame()
all_t_cnt = pd.Series(list(range(max(transactions_in_visits.amount) + 1)), name='transactions_count').to_frame()
result = all_t_cnt.join(transactions_in_visits.groupby(['amount']).agg(visits_count = ('user_id', 'count')), how='left').fillna(0).sort_values(by=['transactions_count'])

In [ ]:
def draw_chart(visits: pd.DataFrame, transactions: pd.DataFrame) -> pd.DataFrame:
    transactions_in_visits = visits.merge(transactions,
                                          how='left',
                                          left_on=['user_id', 'visit_date'],
                                          right_on=['user_id', 'transaction_date'])\
                                          .groupby(['user_id', 'visit_date'], as_index=False)\
                                          .amount.count()
    all_t_cnt = pd.Series(list(range(max(transactions_in_visits.amount) + 1)), name='transactions_count')\
                  .to_frame()
    result = all_t_cnt.join(transactions_in_visits\
                      .groupby(['amount'])\
                      .agg(visits_count = ('user_id', 'count')), how='left')\
                      .fillna(0)\
                      .sort_values(by=['transactions_count'])
    return result

615. Average Salary: Departments VS Company

https://leetcode.com/problems/average-salary-departments-vs-company/description/

```sql
WITH company_avg_salary AS (
SELECT
to_char(pay_date, 'YYYY-MM') AS month,
round(avg(amount), 2) as avg_salary
FROM Salary s
GROUP BY to_char(pay_date, 'YYYY-MM')),

dept_avg_salary AS (
SELECT
e.department_id,
to_char(pay_date, 'YYYY-MM')  AS month,
round(avg(amount), 2) as avg_salary
FROM Salary s
JOIN Employee e USING(employee_id)
GROUP BY department_id , to_char(pay_date, 'YYYY-MM'))

SELECT
month as pay_month,
department_id,
CASE WHEN ds.avg_salary > cs.avg_salary THEN 'higher'
     WHEN ds.avg_salary < cs.avg_salary THEN 'lower'
     ELSE 'same' END AS comparison
FROM dept_avg_salary ds
LEFT JOIN company_avg_salary cs USING(month)
```

In [ ]:
data = [[1, 1, 9000, '2017/03/31'], [2, 2, 6000, '2017/03/31'], [3, 3, 10000, '2017/03/31'], [4, 1, 7000, '2017/02/28'], [5, 2, 6000, '2017/02/28'], [6, 3, 8000, '2017/02/28']]
salary = pd.DataFrame(data, columns=['id', 'employee_id', 'amount', 'pay_date']).astype({'id':'Int64', 'employee_id':'Int64', 'amount':'Int64', 'pay_date':'datetime64[ns]'})
data = [[1, 1], [2, 2], [3, 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'department_id']).astype({'employee_id':'Int64', 'department_id':'Int64'})

In [ ]:
import numpy as np

In [ ]:
def average_salary(salary: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    salary['pay_month'] = salary.pay_date.dt.to_period('M').astype(str)
    comp_avg_salary = salary.groupby(['pay_month'])\
                            .agg(comp_avg_salary = ('amount', 'mean'))\
                            .reset_index()
    dept_avg_salary = salary.merge(employee, how='inner', on='employee_id')\
                            .groupby(['department_id', 'pay_month'])\
                            .agg(dept_avg_salary = ('amount', 'mean'))\
                            .reset_index()
    agg_salaries =  dept_avg_salary.merge(comp_avg_salary, how='inner', on='pay_month')
    agg_salaries['comparison'] = np.where(agg_salaries.dept_avg_salary > agg_salaries.comp_avg_salary,
                                            'higher',
                                            np.where(agg_salaries.dept_avg_salary < agg_salaries.comp_avg_salary,
                                                    'lower',
                                                    'same'))
    return agg_salaries[['pay_month','department_id', 'comparison']]

618. Students Report By Geography

https://leetcode.com/problems/students-report-by-geography/description/

```sql
SELECT
max(name) FILTER (WHERE continent = 'America') as America,
max(name) FILTER (WHERE continent = 'Asia') as Asia,
max(name) FILTER (WHERE continent = 'Europe') as Europe
FROM
(SELECT *,
row_number() OVER (PARTITION BY continent order by name) as rn
FROM Student)
GROUP BY rn
ORDER BY rn
```

In [ ]:
data = [['Jane', 'America'], ['Pascal', 'Europe'], ['Xi', 'Asia'], ['Jack', 'America']]
student = pd.DataFrame(data, columns=['name', 'continent']).astype({'name':'object', 'continent':'object'})

In [ ]:
def geography_report(student: pd.DataFrame) -> pd.DataFrame:
    student = pd.concat([student, pd.DataFrame([[None, 'Europe'], [None, 'Asia']], columns=student.columns)], axis=0)
    student = student.sort_values('name')
    student['order'] = student.groupby('continent').cumcount()
    result = student.pivot(index='order', columns = 'continent', values = 'name').reset_index(drop=True)
    return result.dropna(how='all')

1097. Game Play Analysis V

https://leetcode.com/problems/game-play-analysis-v/description/

```sql
WITH t AS (
SELECT
player_id, event_date,
first_value(event_date) OVER (PARTITION BY player_id  ORDER BY event_date) as install_dt ,
event_date - first_value(event_date) OVER (PARTITION BY player_id  ORDER BY event_date) as ret
FROM Activity a)

SELECT
install_dt,
count(DISTINCT player_id) as installs,
round(count(DISTINCT player_id) FILTER (WHERE ret=1) * 1.0 / count(DISTINCT player_id), 2) as Day1_retention
FROM t
GROUP BY install_dt
```

1127. User Purchase Platform

https://leetcode.com/problems/user-purchase-platform/description/

```sql
WITH analytical AS (
SELECT
    spend_date,
    platform
FROM
(SELECT UNNEST(ARRAY['desktop', 'mobile', 'both']) platform),
(SELECT DISTINCT spend_date  FROM Spending) as spend_date),

t as (
SELECT
    user_id, spend_date,
    sum(amount) as total_amount ,
    count(DISTINCT user_id) as total_users ,
    max(CASE WHEN platform = 'mobile' THEN 1 ELSE 0 END)  +
    max(CASE WHEN platform = 'desktop' THEN 10 ELSE 0 END) AS platform
FROM Spending
GROUP BY user_id, spend_date),


data as (

SELECT
    spend_date,
    CASE WHEN platform = 1 THEN 'mobile' WHEN platform = 10 THEN 'desktop' ELSE 'both' END as platform,
    sum(total_amount) as total_amount,
    sum(total_users) as total_users
FROM t
GROUP BY  spend_date, platform)

SELECT
    a.spend_date,
    a.platform,
    coalesce(sum(d.total_amount), 0) as total_amount,
    coalesce(sum(d.total_users), 0) as total_users
FROM data d
RIGHT JOIN analytical a USING(platform, spend_date)
GROUP BY  a.spend_date, a.platform
```

1194. Tournament Winners

https://leetcode.com/problems/tournament-winners/description/

```sql
WITH all_matches AS (

(SELECT
    first_player as player_id,
    first_score as score
FROM Matches)

UNION ALL

(SELECT
    second_player as player_id,
    second_score  as score
FROM Matches)),

players_ranked AS (
SELECT DISTINCT
    player_id,
    group_id,
    rank() OVER (PARTITION BY group_id ORDER BY sum(score) DESC, player_id)  as rnk
FROM
    all_matches m
INNER JOIN Players p USING (player_id)
GROUP BY player_id, group_id)


SELECT
    group_id,
    player_id
FROM players_ranked
WHERE rnk = 1
```

1225. Report Contiguous Dates

https://leetcode.com/problems/report-contiguous-dates/description/

```sql
WITH t as (
SELECT
  'succeeded' flag,
  success_date as dates     
FROM Succeeded

UNION ALL

SELECT
  'failed',
  fail_date as dates     
FROM Failed ),

t2 as (
SELECT
*,
row_number() OVER (ORDER BY dates) -
dense_rank() OVER (PARTITION BY flag ORDER BY dates)  as delta
FROM t
WHERE dates BETWEEN '2019-01-01' AND '2019-12-31')

SELECT
  flag as period_state ,
  min(dates) as start_date  ,
  max(dates) as end_date
FROM t2
GROUP BY flag,delta
ORDER BY 2
```

1369. Get the Second Most Recent Activity

https://leetcode.com/problems/get-the-second-most-recent-activity/description/

```sql
SELECT
  username,
  activity,
  startDate,
  endDate
FROM (
SELECT
  *,
  row_number() OVER (PARTITION BY username   ORDER BY endDate DESC) as rn,
  count(activity) OVER (PARTITION BY username) as cnt
FROM UserActivity) sub
WHERE sub.rn = 2 or sub.cnt = 1
```

1412. Find the Quiet Students in All Exams

https://leetcode.com/problems/find-the-quiet-students-in-all-exams/description/

```sql
WITH exam_scores AS (
    SELECT
        exam_id,
        MIN(score) AS min_score,
        MAX(score) AS max_score
    FROM Exam
    GROUP BY exam_id
),

quiet_student_ids as (
SELECT
  e.student_id
FROM Exam e
INNER JOIN exam_scores USING(exam_id)
GROUP BY student_id
HAVING count(exam_id) FILTER (WHERE e.score > min_score AND e.score < max_score) = count(exam_id)
)


SELECT
  s.*
FROM quiet_student_ids
INNER JOIN Student s USING(student_id)
```

1479. Sales by Day of the Week

https://leetcode.com/problems/sales-by-day-of-the-week/description/

```sql

WITH weekday_sales AS (
SELECT
  i.item_category,
  to_char(order_date, 'DAY') as weekdays,
  sum(quantity) as items_cnt
FROM Orders
RIGHT JOIN Items i USING(item_id)
GROUP BY i.item_category, to_char(order_date, 'DAY'))


SELECT
  item_category as CATEGORY,
  SUM(CASE WHEN TRIM(weekdays) = 'MONDAY' THEN items_cnt ELSE 0 END) AS MONDAY,
  SUM(CASE WHEN TRIM(weekdays) = 'TUESDAY' THEN items_cnt ELSE 0 END) AS TUESDAY,
  SUM(CASE WHEN TRIM(weekdays) = 'WEDNESDAY' THEN items_cnt ELSE 0 END) AS WEDNESDAY,
  SUM(CASE WHEN TRIM(weekdays) = 'THURSDAY' THEN items_cnt ELSE 0 END) AS THURSDAY,
  SUM(CASE WHEN TRIM(weekdays) = 'FRIDAY' THEN items_cnt ELSE 0 END) AS FRIDAY,
  SUM(CASE WHEN TRIM(weekdays) = 'SATURDAY' THEN items_cnt ELSE 0 END) AS SATURDAY,
  SUM(CASE WHEN TRIM(weekdays) = 'SUNDAY' THEN items_cnt ELSE 0 END) AS SUNDAY
FROM weekday_sales
GROUP BY item_category
ORDER BY 1
```